## Importing Dependencies

In [1]:
import numpy as np
import os
import matplotlib.pyplot as plt
from imutils import paths
import cv2 as cv

from tensorflow.keras.applications import MobileNetV2 
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tensorflow.keras.models import load_model

from imutils.video import VideoStream
import imutils

## Loading Data

In [ ]:
dataset = '/Users/kunal/Desktop/ML projects/dataset'
imagePaths = list(paths.list_images(dataset))

In [ ]:
#imagePaths

## Preprocessing Data

In [ ]:
data = []
labels = []

for i in imagePaths:
    label = i.split(os.path.sep)[-2]
    labels.append(label)
    image = load_img(i, target_size = (224, 224))
    image = img_to_array(image)
    image = preprocess_input(image)
    data.append(image)

In [ ]:
data

In [ ]:
data = np.array(data, dtype = 'float32')
labels = np.array(labels)

In [ ]:
data.shape

In [ ]:
labels

In [ ]:
lb = LabelBinarizer()
labels = lb.fit_transform(labels)
labels = to_categorical(labels)

In [ ]:
labels

In [ ]:
train_X, test_X, train_Y, test_Y = train_test_split(data, labels, test_size = 0.20, random_state = 10, stratify = labels)

In [ ]:
aug = ImageDataGenerator(rotation_range=20, zoom_range= 0.15, width_shift_range= 0.2, shear_range= 0.15, horizontal_flip= True, vertical_flip= True, fill_mode='nearest')

In [ ]:
baseModel = MobileNetV2(weights='imagenet', include_top=False, input_tensor= Input(shape = (224,224,3)))

In [ ]:
baseModel.summary()

In [ ]:
headModel = baseModel.output
headModel = AveragePooling2D(pool_size = (7,7))(headModel)
headModel = Flatten(name = 'Flatten')(headModel)
headModel = Dense(128, activation = 'relu')(headModel)
headModel = Dropout(0.5)(headModel)
headModel = Dense(2, activation = 'softmax')(headModel)

model = Model(inputs = baseModel.input, outputs = headModel)

In [ ]:
for layer in baseModel.layers:
    layer.trainable = False

In [ ]:
model.summary()

In [ ]:
learning_rate = 0.001
Epochs = 20
batch_size = 12

opt = Adam(lr=learning_rate, decay = learning_rate/Epochs)

model.compile(loss = 'binary_crossentropy', optimizer = opt, metrics = ['accuracy'])

H = model.fit(
    aug.flow(train_X, train_Y, batch_size = batch_size),
    steps_per_epoch=len(train_X)//batch_size,
    validation_data=(test_X, test_Y),
    validation_steps=len(test_X)//batch_size,
    epochs = Epochs
)

model.save('mask.model')

In [ ]:
predict = model.predict(test_X, batch_size=batch_size)
predict = np.argmax(predict, axis = 1)
print(classification_report(test_Y.argmax(axis=1), predict, target_names = lb.classes_))

In [2]:
model = load_model('mask.model')

In [ ]:
model.summary()

# Test

In [3]:
prototxtPath = os.path.sep.join(['/Users/kunal/Desktop/ML projects/deploy.prototxt'])
weightsPath = os.path.sep.join(['/Users/kunal/Desktop/ML projects/res10_300x300_ssd_iter_140000.caffemodel'])

In [4]:
faceNet = cv.dnn.readNet(prototxtPath, weightsPath)

In [ ]:
image.shape

In [5]:
def detect_mask(frame, faceNet, model):
    #Take dimensions to make blob
    (h,w) = frame.shape[:2]
    blob = cv.dnn.blobFromImage(frame, 1.0, (300,300), (104.0, 177.0, 123.0))
    
    faceNet.setInput(blob)
    detections = faceNet.forward()
    
    #initialise the list of faces, their correspondng locations and list of predictiob
    faces = []
    locs = []
    preds = []
    
    for i in range(0, detections.shape[2]):
        confidence = detections[0,0,i,2]
        
        if confidence > 0.5:
            #coordinates of x and y
            box = detections[0,0,i,3:7]*np.array([w,h,w,h])
            (startX, startY, endX, endY) = box.astype('int')
            
            #ensure the bounding boxes fall within the dimension of the frame
            (startX, startY) = (max(0, startX), max(0, startY))
            (endX, endY) = (min(w-1, endX), min(h-1, endY))
            
            #convert ROI from BGR to RGB channel, resize ut to 224, 224 and preprocess it
            face = frame[startY:endY, startX:endX]
            face = cv.cvtColor(face, cv.COLOR_BGR2RGB)
            face = cv.resize(face, (224, 224))
            face = img_to_array(face)
            face = preprocess_input(face)
            
            faces.append(face)
            locs.append((startX, startY, endX, endY))
            
            
        if len(faces)>0:
            faces = np.array(faces, dtype = 'float32')
            preds = model.predict(faces, batch_size = 12)
            
        return (locs, preds)

In [6]:
vs = VideoStream(src = 0).start()

while True:
    #resize the frames
    frame = vs.read()
    frane = imutils.resize(frame, width = 400)
    
    #detect faces and mask
    (locs, preds) = detect_mask(frame, faceNet, model)
    
    #loop over detected face 
    for (box, pred) in zip(locs, preds):
        (startX, startY, endX, endY) = box
        (mask, withoutMask) = pred
        
         #color and lobel of the bonding box and text
        label = 'Mask' if mask>withoutMask else 'No Mask'
        color = (0,255, 0) if label == 'Mask' else (0, 0, 255)
            
        #display the label and bonding boxes
        cv.putText(frame, label, (startX, startY-10), cv.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
            
        cv.rectangle(frame,(startX, startY), (endX, endY), 2)
            
    cv.imshow('Frame', frame)
    key = cv.waitKey(1) & 0xFF
    
    if key ==ord('q'):
        break
    
cv.destroyAllWindows
vs.stop()

error: OpenCV(4.5.1) /private/var/folders/nz/vv4_9tw56nv9k3tkvyszvwg80000gn/T/pip-req-build-39p1qqfs/opencv/modules/imgproc/src/color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cvtColor'


In [2]:
import cv2 as cv